# COVID-19 Statistics Notebook

Some simple examples of pandas and jupyter notebook applied to the JHU COVID-19 online reports. 

If you are new to Jupyter, reach up above this box, select "Kernel -> Restart & Run All" to execute all the stuff below.  If you want to add new cells, go to the end of this notebook, hit the "+" button above, and then <SHIFT><RETURN> to evaluate what you typed. 
    
There are good Jupyter tutorials on the web.  This isn't one of them. 

In [ ]:
import pandas as pd
import numpy as np
import LoadDatabase as ldb
import matplotlib.pyplot as plt

From the README.md file at [JHU COVID-19 Dataset](https://raw.github.com/CSSEGISandData/COVID-19)

# JHU CSSE COVID-19 Dataset

## Table of contents

 * [Daily reports (csse_covid_19_daily_reports)](#daily-reports-csse_covid_19_daily_reports)
 * [USA daily state reports (csse_covid_19_daily_reports_us)](#usa-daily-state-reports-csse_covid_19_daily_reports_us)
 * [Time series summary (csse_covid_19_time_series)](#time-series-summary-csse_covid_19_time_series)
 * [Data modification records](#data-modification-records)
 * [UID Lookup Table Logic](#uid-lookup-table-logic)
---

## [Daily reports (csse_covid_19_daily_reports)](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports)

This folder contains daily case reports. All timestamps are in UTC (GMT+0).

### File naming convention
MM-DD-YYYY.csv in UTC.

### Field description
* <b>FIPS</b>: US only. Federal Information Processing Standards code that uniquely identifies counties within the USA.
* <b>Admin2</b>: County name. US only.
* <b>Province_State</b>: Province, state or dependency name.
* <b>Country_Region</b>: Country, region or sovereignty name. The names of locations included on the Website correspond with the official designations used by the U.S. Department of State.
* <b>Last Update</b>: MM/DD/YYYY HH:mm:ss  (24 hour format, in UTC).
* <b>Lat</b> and <b>Long_</b>: Dot locations on the dashboard. All points (except for Australia) shown on the map are based on geographic centroids, and are not representative of a specific address, building or any location at a spatial scale finer than a province/state. Australian dots are located at the centroid of the largest city in each state.
* <b>Confirmed</b>: Confirmed cases include presumptive positive cases  and probable cases, in accordance with CDC guidelines as of April 14.
* <b>Deaths</b>: Death totals in the US include confirmed and probable, in accordance with [CDC](https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/cases-in-us.html) guidelines as of April 14.
* <b>Recovered</b>: Recovered cases outside China are estimates based on local media reports, and state and local reporting when available, and therefore may be substantially lower than the true number. US state-level recovered cases are from [COVID Tracking Project](https://covidtracking.com/).
* <b>Active:</b> Active cases = total confirmed - total recovered - total deaths.
* <b>Incidence_Rate</b>: Admin2 + Province_State + Country_Region.
* <b>Case-Fatality Ratio (%)</b>: = confirmed cases per 100,000 persons.
* <b>US Testing Rate</b>: = total test results per 100,000 persons. The "total test results" is equal to "Total test results (Positive + Negative)" from [COVID Tracking Project](https://covidtracking.com/).
* <b>US Hospitalization Rate (%)</b>: = Total number hospitalized / Number confirmed cases. The "Total number hospitalized" is the "Hospitalized – Cumulative" count from [COVID Tracking Project](https://covidtracking.com/). The "hospitalization rate" and "hospitalized - Cumulative" data is only presented for those states which provide cumulative hospital data.

### Update frequency
* Files on and after April 23, once per day between 03:30 and 04:00 UTC.
* Files from February 2 to April 22: once per day around 23:59 UTC.
* Files on and before February 1: the last updated files before 23:59 UTC. Sources: [archived_data](https://github.com/CSSEGISandData/COVID-19/tree/master/archived_data) and dashboard.

### Data sources
Refer to the [mainpage](https://github.com/CSSEGISandData/COVID-19).




In [ ]:
# Read the US daily report.  
# Choices are daily_reports (worldwide), daily_reports_us (US only)
daily_rep_us = ldb.loadDatabase("US")
daily_rep_us.head(3)

#### What do the statistics for Maryland look like?

In [ ]:
daily_rep_us.loc[daily_rep_us["Province_State"] == "Maryland"]

### Let's create a new column showing the proportion of deaths vs all "outcomes" (patient recovers, or dies)

In [ ]:
# An outcome here is defined as either a reported death or reported recovery
daily_rep_us["Deaths_vs_Outcomes"] = np.divide(daily_rep_us["Deaths"], np.add(daily_rep_us["Deaths"], daily_rep_us["Recovered"]))

In [ ]:
# now sort by the probability of death vs. outcome
daily_rep_us.sort_values(by="Deaths_vs_Outcomes", ascending=False, inplace=True)

In [ ]:
# Note that in Nevada, there are more death outcomes than recovered outcomes.
daily_rep_us.head(10)

### Let's plot the top 10

In [ ]:
dvs = 'Deaths_vs_Outcomes'
dru = daily_rep_us.copy()
ax = dru.dropna(subset=[dvs]).sort_values(by=dvs, ascending=False).head(10).plot.barh(x='Province_State', y=dvs, title="Deaths vs. Outcomes for States Reporting Recoveries and Deaths", figsize=(10,10))

### What about the states that are relaxing closure rules?

In [ ]:
open_lis = ["Arizona", "Alabama", "South Carolina"]
daily_rep_us[daily_rep_us["Province_State"].isin(open_lis)]

In [ ]:
daily_rep_us["Population (M)"] = 0.1 * np.divide(daily_rep_us["People_Tested"],daily_rep_us["Testing_Rate"])
daily_rep_us['Deaths per Thousand'] = 0.001 * np.divide(daily_rep_us["Deaths"], daily_rep_us["Population (M)"])
daily_rep_us.sort_values(by='Deaths per Thousand', inplace=True, ascending=False)
daily_rep_us.head(10)

In [ ]:
fig = plt.figure(figsize=[10,20])
ax = daily_rep_us.plot.barh(x='Province_State', y='Deaths per Thousand', figsize=(10,20))